# 1. Setup

## 1.1 Import Dependencies

In [ ]:
# Import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt
# plt.imshow() --> Main usage of matplotlib here

In [ ]:
# Import tensorflow dependencies - Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [ ]:
# The Layer class helps us to define a custom layer
# Con2D --> Convolution 
# Dense --> Fully connected Layer
# MaxPooling2D --> Perform Maxpooling
# Input --> Base class(Defines What we are going to pass through to our model and our layer/model compile that all together )
# Flatten --> Takesall the info from the previous layer and flattens it down to a single dimension (CNN output(2D array) --> Desnse layer(1D array))

## 1.2 Set GPU Growth

In [ ]:
# By default, Tensorflow allocates all available GPU memory for the training process, which can cause the system to run out of memory if the model is large or the GPU has limited memory.
# setting the memory growth option to True ensures that Tensorflow will allocate GPU memory dynamically, rather than allocating all of it upfront. 

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

## 1.4 Create Folder Structures

#### 1.4.1 This is for Windows

In [ ]:
# Setup paths
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [ ]:
# Make the directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

#### 1.4.2 This is for Kaggle

In [ ]:
# input image --> Anchor image
# Verification image(which is similar) --> +ve image
# Verification image(which is not similar) --> -ve image
# -ve images --> labeled faces(repo)
# Setup paths
POS_PATH = '//kaggle//input//positive'
NEG_PATH = '//kaggle//input//negative'
ANC_PATH = '//kaggle//input//anchor'

In [ ]:
print(f"Folder path for +ve image :{POS_PATH}")
print(f"Folder path for +ve image :{NEG_PATH}")
print(f"Folder path for +ve image :{ANC_PATH}")

# 2. Collect Positives and Anchors

In [ ]:
# http://vis-www.cs.umass.edu/lfw/
# Uncompress Tar GZ Labelled Faces in the Wild Dataset
!tar -xf /kaggle/input/labelled-faces-in-the-wild/lfw.tgz

In [ ]:
# Move LFW Images to the following repository data/negative
for directory in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw', directory)):
        EX_PATH = os.path.join('lfw', directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)

## 2.2 Collect Positive and Anchor Classes

In [ ]:
# Import uuid library to generate unique image names
import uuid
os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))

In [ ]:
# Establish a connection to the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    ret, frame = cap.read()
   
    # Cut down frame to 250x250px as we have to train it accordingly to negative
    # frame = frame[:250,:250,:] --> We didn't took this as it's cutting image from 
    # the starting and in the camera we are facing some where in the middle normally    
    # 120 is near middle of the image and we went till 120+250 and 200 + 250 as we want something in betweem    
    frame = frame[120:120+250,200:200+250, :]
    
    
    # Collect anchors 
    if cv2.waitKey(1) & 0XFF == ord('a'):
        # Create the unique file path 
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out anchor image
        cv2.imwrite(imgname, frame)
    
    # Collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Create the unique file path 
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        
        # Write out positive image
        cv2.imwrite(imgname, frame)
    
    # Show image back to screen what it's tracking
    cv2.imshow('Image Collection', frame)
    
    # Breaking gracefully
    # Here the screen is stopped for 1 millisecond and the image is grabbed in that time period only    
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()

In [ ]:
frame.shape

In [ ]:
plt.imshow(frame[:,:, :])

# 3. Load and Preprocess Images

## 3.1 Get Image Directories

In [ ]:
#  Grab all the different mages in a particular directory
#  Creates a data generator / using a generator then loop through and grab all the files within that specific dictionary / specific directory
#  Grabs all the specific files with .jpg extension in the back
#  Helps in batch processing will implement batchwise
anchor = tf.data.Dataset.list_files(ANC_PATH+'/*.jpg').take(300)
positive = tf.data.Dataset.list_files(POS_PATH+'/*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEG_PATH+'/*.jpg').take(300)

In [ ]:
# This is basically making a itterator to itterate over the anchor image files
dir_test = anchor.as_numpy_iterator()

In [ ]:
# Full path to a specific image within a particular directory
# It's an itterator
print(dir_test.next())

## 3.2 Preprocessing - Scale and Resize

### Need of Scaling
* Machine learning algorithms and Deep learning models require the input data to be normalized, which means that the values are scaled to a specific range, often [0, 1] or [-1, 1].
* Normalizing the image to the range [0, 1] increases the memory usage by a factor of 4.
* Scaling the image to the range [0, 1] before storing it, we can avoid the need to divide each pixel value by 255 during training,

In [ ]:
def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    print("Before Preprocessing")
    print(img)
    
    # Preprocessing steps - resizing the image to be 100x100x3(3 is beacuse of RGB seperate matrixes)
    # It do    
    
    # We are resizing it as in the paper it's written it should be 105x 105
    img = tf.image.resize(img, (100,100))
    print(img)
    # Scale image to be between 0 and 1 
    img = img / 255.0
    
    print("After Preprocessing")
    print(img)
    # Return image
    return img

In [ ]:
def withoutpreprocessing(file_path):
     # Read in image from file path(in bytes)
    byte_img = tf.io.read_file(file_path)
    
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    img = tf.image.resize(img, (100,100))
    return img

In [ ]:
# This works only for non scaled image as non scaled image will give same output when used with plt.imshow though the values lie in the range of 0-1
def check_images_same(image1,image2):
    difference = np.abs(image1.numpy()- image2.numpy())
    if np.sum(difference) == 0:
        return True
    else:
        return False

##### 3.2.1 Image after preprocessing

In [ ]:
img = preprocess('//kaggle//input//anchor/4a74e410-ca6e-11ed-a96a-f47b0953a0e7.jpg')

##### 3.2.2 Checking whether scaling os apropriately performed or not

In [ ]:
print(f"Maximum pixel val: {img.numpy().max()}\nMinimum pixel val:{img.numpy().min()}")

In [ ]:
# plt.imshow() expects pixel values to be in the range of [0, 255], so it automatically 
# applies a normalization step to the pixel values of the input image(for viewing purpose only) 
# else the image looks darker and has lower contrast.
plt.imshow(img)

##### 3.2.3 Image without preprocessing

In [ ]:
# It will show same image as it automatically scales it back to [0,255]
plt.imshow(withoutpreprocessing('//kaggle//input//anchor/4a74e410-ca6e-11ed-a96a-f47b0953a0e7.jpg'))

In [ ]:
# It's comming out to be false as we are comparing pixel by pixel
check_images_same(img,withoutpreprocessing('//kaggle//input//anchor/4a74e410-ca6e-11ed-a96a-f47b0953a0e7.jpg'))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Load image as a numpy array in the range [0, 1]
image = preprocess('//kaggle//input//anchor/4a74e410-ca6e-11ed-a96a-f47b0953a0e7.jpg')

# Display image without automatic scaling to [0, 255]
fig, ax = plt.subplots()
im = ax.imshow(image, vmin=0, vmax=1) # cmap='gray' for grayscale images
fig.colorbar(im)
plt.show()


## 3.3 Create Labelled Dataset

In [ ]:
#       INPUTS           OUTPUTS
# (anchor, positive) => 1,1,1,1,1
# (anchor, negative) => 0,0,0,0,0

In [ ]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))

# Data is the dataset
data = positives.concatenate(negatives)

In [ ]:
# Runnng an itterator over the data
samples = data.as_numpy_iterator()
example = samples.next()

# 3.4 Build Train and Test Partition

In [ ]:
# Making a preprocess_twin function to apply preprocessing function over the two images
def preprocess_twin(input_img, validation_img, label):
    print(type(input_img),type(validation_img),type(label))
    return(preprocess(input_img), preprocess(validation_img), label)

In [ ]:
# Checking whether it's working correctly
res = preprocess_twin(*example)

#### 3.4.1 This is for checking whether the preprocess_twin function working correctly 

In [ ]:
# Input image
plt.imshow(res[1])

In [ ]:
# Validation image
plt.imshow(res[0])

In [ ]:
# Label for the above two images
res[2]

### Code for determining the batch size appropriate for the particular machine

In [ ]:
# Build dataloader pipeline
# Appling preprocess function to each row of the data
data = data.map(preprocess_twin)

# Clearing the cache
data = data.cache()

# We shuffling it up
data = data.shuffle(buffer_size=1024)

In [ ]:
# check the length of data

In [ ]:
# Training partition taking 70% data for training purposes
train_data = data.take(round(len(data)*.7))

#  create batches of data with a batch size of 16. This means that the 
# dataset will be divided into chunks of 16 samples each, and each chunk 
# will be fed to the machine learning or deep learning model as one batch 
# during training.
train_data = train_data.batch(16)

# Prefetching
train_data = train_data.prefetch(8)

#### USE OF PRE-FETCHING
*  enables pre-fetching of a specified number of elements from the dataset.
* improve the training performance of machine learning models by reducing the idle time between iterations. 
* improve the training performance of machine learning models by reducing the idle time between iterations. 
* In general, a larger number of elements can improve training performance by reducing the idle time between batches. However, prefetching too many elements can lead to memory overflow or slow down the training process.
* A good starting point is to set the prefetch buffer size to the number of elements that can be processed by the model in one iteration, also known as the batch size. For example, if the batch size is set to 32, you can set the prefetch buffer size to 32 or a multiple of 32, such as 64 or 128.

In [ ]:
# Testing partition

# This will skip first 70% data
test_data = data.skip(round(len(data)*.7))


test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In general, a larger number of elements can improve training performance by reducing the idle time between batches. However, prefetching too many elements can lead to memory overflow or slow down the training process.

A good starting point is to set the prefetch buffer size to the number of elements that can be processed by the model in one iteration, also known as the batch size. For example, if the batch size is set to 32, you can set the prefetch buffer size to 32 or a multiple of 32, such as 64 or 128.

# 4. Model Engineering

## 4.1 Build Embedding Layer

In [ ]:
def make_embedding(): 
    inp = Input(shape=(100,100,3), name='input_image')
    
    # First block
    c1 = Conv2D(64, (10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)
    
    # Second block
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    print(c2)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)
    
    # Third block 
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)
    
    # Final embedding block
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)
    
    
    return Model(inputs=[inp], outputs=[d1], name='embedding')

In [ ]:
embedding = make_embedding()

In [ ]:
embedding.summary()

## 4.2 Build Distance Layer

In [ ]:
# Siamese L1 Distance class
# Defining custom nerual network class
# It's inheriting Layer class
class L1Dist(Layer):
    
    # Init method - inheritance
    def __init__(self, **kwargs):
        super().__init__()
       
    # Magic happens here - similarity calculation
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [ ]:
l1 = L1Dist()

## 4.3 Make Siamese Model

In [ ]:
def make_siamese_model(): 
    
    # Anchor image input in the network
    input_image = Input(name='input_img', shape=(100,100,3))
    
    # Validation image in the network 
    validation_image = Input(name='validation_img', shape=(100,100,3))
    
    # Combine siamese distance components
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))
    
    # Classification layer 
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [ ]:
input_image = Input(name='input_img', shape=(100,100,3))
validation_image = Input(name='validation_img', shape=(100,100,3))

In [ ]:
inp_embedding = embedding(input_image)
val_embedding = embedding(validation_image)

In [ ]:
siamese_layer = L1Dist()

In [ ]:
distances = siamese_layer(inp_embedding, val_embedding)

In [ ]:
classifier = Dense(1, activation='sigmoid')(distances)

In [ ]:
classifier

In [ ]:
siamese_network = Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [ ]:
siamese_network.summary()

In [ ]:
siamese_model = make_siamese_model()

In [ ]:
siamese_model.summary()

SET UP LOSS FUNCTION AND OPTIMISER

In [ ]:
binary_cross_loss = tf.losses.BinaryCrossentropy()

In [ ]:
opt = tf.keras.optimizers.Adam(1e-4) # 0.0001

Checkpoints

In [ ]:
checkpoint_dir = '.\training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

In [ ]:
# getting a batch of 16 rows via firstly converting it as a numpy_iterator
test_batch = train_data.as_numpy_iterator()

In [ ]:
# Geting the batch
batch_1 = test_batch.next()

In [ ]:
# our valiation and (positive/negative images) are at index 1 and 0 resp
X = batch_1[:2]

In [ ]:
# This is for label
y = batch_1[2]

In [ ]:
y

In [ ]:
tf.losses.BinaryCrossentropy??

In [ ]:
@tf.function
def train_step(batch):
    
    # Record all of our operations 
    with tf.GradientTape() as tape:     
        # Get anchor and positive/negative image
        X = batch[:2]
        # Get label
        y = batch[2]
        
        # Forward pass
        yhat = siamese_model(X, training=True)
        # Calculate loss
        loss = binary_cross_loss(y, yhat)
    print(loss)
        
    # Calculate gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    
    # Calculate updated weights and apply to siamese model
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
        
    # Return loss
    return loss

In [ ]:
# Import metric calculations
from tensorflow.keras.metrics import Precision, Recall

In [ ]:
def train(data, EPOCHS):
    # Loop through epochs
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        # Creating a metric object 
        r = Recall()
        p = Precision()
        
        # Loop through each batch
        for idx, batch in enumerate(data):
            # Run train step here
            loss = train_step(batch)
            yhat = siamese_model.predict(batch[:2])
            r.update_state(batch[2], yhat)
            p.update_state(batch[2], yhat) 
            progbar.update(idx+1)
        print(loss.numpy(), r.result().numpy(), p.result().numpy())
        
        # Save checkpoints
        if epoch % 10 == 0: 
            checkpoint.save(file_prefix=checkpoint_prefix)

In [ ]:
EPOCHS = 50
train(train_data, EPOCHS)

# 6. Evaluate the Model

#### 6.1 Import Metrics

In [ ]:
# Import metric calculations
from tensorflow.keras.metrics import Precision, Recall

In [ ]:
# Get a batch of test data
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [ ]:
y_hat = siamese_model.predict([test_input, test_val])

In [ ]:
# Post processing the results 
[1 if prediction > 0.5 else 0 for prediction in y_hat ]

In [ ]:
y_true

#### 6.3 Calculate Metrics

In [ ]:
# Creating a metric object 
m = Recall()

# Calculating the recall value 
m.update_state(y_true, y_hat)

# Return Recall Result
m.result().numpy()

In [ ]:
# Creating a metric object 
m = Precision()

# Calculating the recall value 
m.update_state(y_true, y_hat)

# Return Recall Result
m.result().numpy()

In [ ]:
r = Recall()
p = Precision()

for test_input, test_val, y_true in test_data.as_numpy_iterator():
    yhat = siamese_model.predict([test_input, test_val])
    r.update_state(y_true, yhat)
    p.update_state(y_true,yhat) 

print(r.result().numpy(), p.result().numpy())

#### 6.4 Viz Results

In [ ]:
# Set plot size 
plt.figure(figsize=(10,8))

# Set first subplot
plt.subplot(1,2,1)
plt.imshow(test_input[0])

# Set second subplot
plt.subplot(1,2,2)
plt.imshow(test_val[0])

# Renders cleanly
plt.show()

# 7. Save Model

In [ ]:
# Save weights
siamese_model.save('siamesemodelv2.h5')

In [ ]:
L1Dist

In [ ]:
# Reload model 
siamese_model = tf.keras.models.load_model('siamesemodelv2.h5', 
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [ ]:
# Make predictions with reloaded model
siamese_model.predict([test_input, test_val])

In [ ]:
# View model summary
siamese_model.summary()